In [1]:
import pandas as pd

In [2]:
kskqkn = pd.read_csv("../1.데이터셋/부실/상장_부실_라벨링하기전.csv", index_col=0)
audit = pd.read_csv("../1.데이터셋/부실/비상장_부실_라벨링하기전.csv", index_col=0)
audit.head(3)

,회사명,거래소코드,회계년도,감사의견코드,[U01410000]영업활동으로 인한 현금흐름(*)(천원),이자보상배율(이자비용),감사부실,이보배부실,현금흐름부실
3,(유)가든주류,97082,2022,DU,-26425950.0,-1.44,1.0,NaN,NaN
37,(주)ATS엔지니어링,61907,2016,DU,1286292.0,-16.37,1.0,NaN,NaN
123,(주)HM금속,43596,2015,DU,-518828.0,-1.65,1.0,NaN,NaN


In [3]:
kskqkn = kskqkn.sort_values(by=["거래소코드", "회계년도"], ascending=True)
audit = audit.sort_values(by=["거래소코드", "회계년도"], ascending=True)

In [4]:
# 상장 부실판단
kskqkn["부실판단"] = None
cond1 = (kskqkn["감사부실"] == 1) | (kskqkn["이보배부실"] == 1) | (kskqkn["현금흐름부실"] == 1)
cond2 = (~cond1) & (kskqkn["감사부실"] == 0) | (kskqkn["이보배부실"] == 0) | (kskqkn["현금흐름부실"] == 0)

kskqkn.loc[cond1, "부실판단"] = 1
kskqkn.loc[cond2, "부실판단"] = 0

In [5]:
## 상장기업
# 부실기업 데이터만 출력
bank1 = kskqkn[kskqkn["부실판단"] == 1]
bank1.drop_duplicates(subset="거래소코드", keep="first", inplace=True)

# 정상기업 데이터만 남기기
bank1_code = bank1["거래소코드"].unique()  # 부실로 선정된 기업의 거래소코드
non_bank1 = kskqkn[~kskqkn["거래소코드"].isin(bank1_code)]  # 부실선정 기업 제거
non_bank1.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

# 부실기업과 정상기업 데이터프레임 합치기
final1 = pd.concat([bank1, non_bank1])
final1.reset_index(drop=True, inplace=True)

/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_35991/919437452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank1.drop_duplicates(subset="거래소코드", keep="first", inplace=True)
/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_35991/919437452.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_bank1.drop_duplicates(subset="거래소코드", keep="last", inplace=True)


In [6]:
# 비상장 부실판단
audit["부실판단"] = None
cond1 = (audit["감사부실"] == 1) | (audit["이보배부실"] == 1) | (audit["현금흐름부실"] == 1)
cond2 = (~cond1) & (audit["감사부실"] == 0) | (audit["이보배부실"] == 0) | (audit["현금흐름부실"] == 0)

audit.loc[cond1, "부실판단"] = 1
audit.loc[cond2, "부실판단"] = 0

In [7]:
### 비상장 기업
# 부실기업 데이터만 출력
bank2 = audit[audit["부실판단"] == 1]
bank2.drop_duplicates(subset="거래소코드", keep="first", inplace=True)

# 정상기업 데이터만 남기기
bank2_code = bank2["거래소코드"].unique()  # 부실로 선정된 기업의 거래소코드
non_bank2 = audit[~audit["거래소코드"].isin(bank2_code)]  # 부실선정 기업 제거
non_bank2.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

# 부실기업과 정상기업 데이터프레임 합치기
final2 = pd.concat([bank2, non_bank2])
final2.reset_index(drop=True, inplace=True)

/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_35991/3228439298.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank2.drop_duplicates(subset="거래소코드", keep="first", inplace=True)
/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_35991/3228439298.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_bank2.drop_duplicates(subset="거래소코드", keep="last", inplace=True)


In [8]:
final1 = final1[["회사명", "거래소코드", "회계년도", "부실판단"]]
final2 = final2[["회사명", "거래소코드", "회계년도", "부실판단"]]

In [9]:
final1.isna().sum()

회사명       0
거래소코드     0
회계년도      0
부실판단     23
dtype: int64

In [10]:
final2.isna().sum()

회사명        0
거래소코드      0
회계년도       0
부실판단     428
dtype: int64

In [11]:
len(final1)

988

In [18]:
len(final1['거래소코드'].unique())

988

In [20]:
len(final2)

33116

In [22]:
final1["부실판단"].value_counts()

부실판단
1    529
0    436
Name: count, dtype: int64

In [21]:
final2['부실판단'].value_counts()

부실판단
0    19596
1    13092
Name: count, dtype: int64

In [13]:
final1.to_csv("../1.데이터셋/Bankruptcy_Label/상장_부실판단.csv")
final2.to_csv("../1.데이터셋/Bankruptcy_Label/비상장_부실판단.csv")